## Content based filtering door natural language processing

In deze notebook zullen een recommender system bouwen door gebruik te maken van Content based filtering door natural language processing.

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import spacy
nlp = spacy.load("en_core_web_md")

def load_jsons(data_path, file):
    """ helper function to load '.json' files (they're not proper jsons) """
    file_path = data_path + file
    with open(file_path) as jsons:
        lines = [json.loads(json_line) for json_line in jsons]
    return pd.DataFrame(lines)

def split_data(data, d = 0.75):
    """Split data in a training and test set.
    
    Arguments:
    data -- any dataFrame.
    d    -- the fraction of data in the training set
    """
    np.random.seed(seed=5)
    mask_test = np.random.rand(data.shape[0]) < d
    return data[mask_test], data[~mask_test]

We laden de data van de aurora dataset in

In [10]:
# Load data for town called Ambridge
# data_path = './data/ambridge/'
data_path = './data/aurora/'
review_file = 'review.json'
business_file = 'business.json'
user_file = 'user.json'
tip_file = 'tip.json'
checkin_file = 'checkin.json'

reviews_total = load_jsons(data_path, review_file)
reviews, reviews_test = split_data(reviews_total, d=0.75)
businesses = load_jsons(data_path, business_file)
users = load_jsons(data_path, user_file)
tips = load_jsons(data_path, tip_file)
checkins = load_jsons(data_path, checkin_file)

In [11]:
display(reviews.head())
display(businesses.head())
display(users.head())
display(tips.head())
display(checkins.head())
businesses.shape, reviews.shape

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,C_7wJ2olzhG1-b3WqriFSA,k5YkkqR1KNWNPKEIzQaEmA,d-u-4Y0BJpbKnEd2NtXAHA,1.0,0,0,0,I recently moved to Aurora from Texas and I al...,2018-07-26 11:49:44
2,JMYC9dVWFSbrP3eZfRiqCQ,yVUJQIv8sBeDx1r0WO94pw,DoWLGxBbR24_0r7mpGg__w,1.0,4,0,0,First you'll notice there is nobody eating ins...,2012-12-04 20:15:25
4,zQdE16AsG8oopDX-jfZD4Q,_rx85IOCAui7IlAf9DClxQ,YB5bR4QQ4mkFPxZwbTpa_Q,4.0,0,0,0,"Home Depot is a convenient, big box supplier. ...",2014-10-27 21:47:17
5,epwaNQV1OO9GAJ1MsctKcA,HB69A6SbcIG97L4Ccn1ucw,UuHT4mGH2E0cFrG-RaYfaw,3.0,0,0,0,Going downhill as of late. My family has been ...,2017-08-29 03:38:49
7,7tktJmmCEHw0oSVaWskjnA,isBw_3wpcmiVBvd7IH9bYA,GeqE_wihijQ2Bf1KCoabKw,4.0,2,0,0,"Love it. Clean restrooms, great playgrounds fo...",2017-02-19 22:39:51


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,JCMnL6-pbWmG7xSVkdBuoQ,Little Caesars Pizza,126 Wellington Street West,Aurora,ON,L4G 2N9,43.998803,-79.474695,2.5,3,1,"{'RestaurantsReservations': 'False', 'Alcohol'...","Restaurants, Pizza",None
1,G3OeHF5jrHvNqi4o3aNezQ,Edward Street Bistro,14785 Yonge Street,Aurora,ON,L4G 1N1,43.987720,-79.464540,3.0,14,0,"{'RestaurantsTakeOut': 'True', 'RestaurantsDel...","Restaurants, Bistros","{'Monday': '17:0-0:0', 'Tuesday': '17:0-0:0', ..."
2,KSKy6-KujlMkwjxlFvMzJw,Aurora Eye Care,220 S Chillicothe Rd,Aurora,OH,44202,41.311173,-81.345960,5.0,7,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Health & Medical, Optometrists, Shopping, Eyew...","{'Monday': '9:0-17:0', 'Tuesday': '10:0-19:0',..."
3,DoWLGxBbR24_0r7mpGg__w,Angelo's Pizzeria & Restaurant,324 Aurora Commons Cir,Aurora,OH,44202,41.315022,-81.357204,3.5,35,1,"{'OutdoorSeating': 'False', 'Alcohol': 'u'beer...","Restaurants, Food, Desserts, Sandwiches, Pizza","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
4,Xh0GgOn6LjfYJJUZQsjDTg,Samurai Japanese Steak House,7155 N Aurora Rd,Aurora,OH,44202,41.353180,-81.386346,3.5,19,1,"{'RestaurantsPriceRange2': '2', 'WiFi': 'u'no'...","Japanese, Restaurants",None


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,NJqmAzerpbAIC0-VBclwgA,Kim,400,2011-01-06 19:09:05,941,349,508,"2012,2013,2014,2015,2016,2017,2018","aAPG4k9TrIVmAGCVUgfQnQ, tN2ml7LWTM2AcZz6JI7ZuQ...",54,...,9,7,0,0,57,79,88,88,36,11
1,PEjS7EYRg1cfHyx3r-w0qg,Maggie,155,2016-07-11 20:27:02,233,98,137,"2016,2017,2018","M0lDdop-4GsZ0Cp_6SyDWA, M-OtWnA7QWWAecAqLfBnJw...",16,...,6,0,0,0,6,4,4,4,18,1
2,DIqmYwkcPFcYnEpWvzXeOQ,Julie,261,2013-09-21 14:10:24,592,333,252,"2014,2015,2016,2017,2018","nOr3_aq60yivqgcso_iDMw, 5-W_iC49mFrevmSL7xnvrg...",32,...,6,0,3,0,19,18,15,15,16,4
3,8KeHUYvc5HVzex6XhP6Fug,Nick,32,2014-08-26 02:32:14,26,3,7,,"Kn7EmDkYekyjl3-YTEq0vQ, Z_FzGGu5vggFwelcMy17oA...",1,...,0,0,0,0,0,0,0,0,0,0
4,nyl_1VcRIAyI55bb_scpdw,Mary,965,2014-08-24 02:43:26,2009,795,1603,"2014,2015,2016,2017,2018","keLUgL_4y60BkppiAsIk8Q, peVCDWIUK1qX5tScxuQ07Q...",34,...,1,2,0,0,21,40,132,132,20,129


,user_id,business_id,text,date,compliment_count
0,_kpe0jAY6ShwvtZPXGI1TA,OTir3s3_WkoBP1MCkMf4Mw,"Hands down, best nail salon on the east side.",2015-05-26 14:37:50,0
1,Mo7i9Mf9B33SveAIChMFNw,FB2zcyp_z1XEM5WngoO5Lg,"Excellent food, fresh fish, good service, with...",2015-01-17 20:44:03,0
2,CxDOIDnH8gp9KXzpBHJYXw,aeROhG8M_G-W5L6XhD3TXA,Weekday dim sum 15% off before 11 am,2013-02-03 16:44:17,0
3,mdmFPUq98OtP8BQ4fCs8wA,86Nva-2rMDMTEtjUZx5toQ,"If you're hungry, order a footlong.",2014-08-28 00:19:08,0
4,mdmFPUq98OtP8BQ4fCs8wA,8IhwL447alJjbvIOtIPqBg,Awesome deli section. Go on off-peak hours for...,2014-08-24 21:52:55,0


,business_id,date
0,-2mTrGw66Sg-ipomugRlmw,"2014-12-21 03:55:09, 2014-12-24 01:50:03, 2015..."
1,-JXt_PElFkCvgRViW05bew,"2011-11-30 18:52:30, 2012-01-25 19:54:22, 2012..."
2,-SzaOJH2Zi1soNWfGDBSAw,"2011-12-31 21:58:13, 2012-05-09 23:35:40, 2013..."
3,-atM342fpLyVYlTDArNqaQ,2017-05-24 15:12:02
4,-dDadUS-WWxOo94KNP73zg,"2011-07-05 00:56:57, 2012-07-05 00:58:57, 2013..."


((356, 14), (3756, 9))

In [12]:
business_reviews = businesses[['business_id', 'categories']].copy()
business_reviews['text'] = '' 

def to_list(cats):
    if isinstance(cats, str):
        return cats.lower().split(', ')
    else:
        return []

business_reviews['categories'] = business_reviews['categories'].map(to_list)
business_reviews = business_reviews[['business_id', 'text', 'categories']]

business_reviews.head()

,business_id,text,categories
0,JCMnL6-pbWmG7xSVkdBuoQ,,"[restaurants, pizza]"
1,G3OeHF5jrHvNqi4o3aNezQ,,"[restaurants, bistros]"
2,KSKy6-KujlMkwjxlFvMzJw,,"[health & medical, optometrists, shopping, eye..."
3,DoWLGxBbR24_0r7mpGg__w,,"[restaurants, food, desserts, sandwiches, pizza]"
4,Xh0GgOn6LjfYJJUZQsjDTg,,"[japanese, restaurants]"


**Alle reviews text tokenization en TF-IDF**

We gaan ervanuit dat de reviews wat kunnen zeggen over categorieen van een business. We verzamelen daarom alle reviews en tips die bij een business horen. We tokenizen alle text en berekenen de TF-IDF van de tokens. Hiermee proberen we extra categorieen toe te voegen aan een business.

In [13]:
for i in business_reviews.index:
    for j in reviews[reviews.business_id == business_reviews.loc[i, 'business_id']]['text']:
        business_reviews.loc[i, 'text'] = str(business_reviews.loc[i, 'text']) + " " + str(j)
    for k in tips[tips.business_id == business_reviews.loc[i, 'business_id']]['text']:
        business_reviews.loc[i, 'text'] = str(business_reviews.loc[i, 'text']) + " " + str(j)
    
business_reviews['text_len'] = business_reviews.text
business_reviews['text_len'] = business_reviews['text_len'].map(lambda x: len(x))

business_reviews['text_tokens'] = business_reviews.text
business_reviews.text_tokens = business_reviews.text_tokens.map(lambda x: nlp(x.lower()))       

business_reviews.head()

,business_id,text,categories,text_len,text_tokens
0,JCMnL6-pbWmG7xSVkdBuoQ,"We had the ""hot-n-ready"" supreme and it was a...","[restaurants, pizza]",838,"( , we, had, the, "", hot, -, n, -, ready, "", s..."
1,G3OeHF5jrHvNqi4o3aNezQ,"I love this place! First, it's around the co...","[restaurants, bistros]",14546,"( , i, love, this, place, !, , first, ,, it, ..."
2,KSKy6-KujlMkwjxlFvMzJw,Eyes. You need them to see. Shocking statisti...,"[health & medical, optometrists, shopping, eye...",2066,"( , eyes, ., you, need, them, to, see, ., shoc..."
3,DoWLGxBbR24_0r7mpGg__w,First you'll notice there is nobody eating in...,"[restaurants, food, desserts, sandwiches, pizza]",11189,"( , first, you, 'll, notice, there, is, nobody..."
4,Xh0GgOn6LjfYJJUZQsjDTg,The hibachi show is the highlight. My family ...,"[japanese, restaurants]",10697,"( , the, hibachi, show, is, the, highlight, .,..."


In [14]:
def lemmas(doc):
    """
    Returns a list of lemmas 
    """
    return [word.lemma_ for word in doc if word.is_alpha and not (word.is_punct or word.is_space or word.is_stop)]

def bag_of_words(doc):
    """
    Takes a spaCy doc and returns a bag of words.
    Does not include any punctuation, whitespace or stop words in the resulting bag.
    """
    # TODO
    
    word_list = lemmas(doc)
    word_count = {}
    
    for i in word_list:
        if(i in word_count):
            word_count[i] += 1
        else:
            word_count[i] = 1
            
    return pd.Series(word_count)

In [18]:
business_reviews['bags'] = business_reviews.text_tokens
business_reviews.bags = business_reviews.bags.map(lambda x: bag_of_words(x))
business_reviews.head()

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,business_id,text,categories,text_len,text_tokens,bags
0,JCMnL6-pbWmG7xSVkdBuoQ,"We had the ""hot-n-ready"" supreme and it was a...","[restaurants, pizza]",838,"( , we, had, the, "", hot, -, n, -, ready, "", s...",-PRON- 15 have 1 the ...
1,G3OeHF5jrHvNqi4o3aNezQ,"I love this place! First, it's around the co...","[restaurants, bistros]",14546,"( , i, love, this, place, !, , first, ,, it, ...",i 75 love 1 this 3...
2,KSKy6-KujlMkwjxlFvMzJw,Eyes. You need them to see. Shocking statisti...,"[health & medical, optometrists, shopping, eye...",2066,"( , eyes, ., you, need, them, to, see, ., shoc...",eye 3 -PRON- 29 need ...
3,DoWLGxBbR24_0r7mpGg__w,First you'll notice there is nobody eating in...,"[restaurants, food, desserts, sandwiches, pizza]",11189,"( , first, you, 'll, notice, there, is, nobody...",first 1 -PRON- 140 notice ...
4,Xh0GgOn6LjfYJJUZQsjDTg,The hibachi show is the highlight. My family ...,"[japanese, restaurants]",10697,"( , the, hibachi, show, is, the, highlight, .,...",the 132 hibachi 26 show ...


In [17]:
def term_frequency(bag):
    """Takes in a bag of words, returns the term frequency of every term in bag as a Series."""
    # TODO
    td_dict = {}
    
    for i in bag.index:
        td_dict[i] = bag[i] / bag.sum()
        
    return pd.Series(td_dict)

def inverse_document_frequency(bags):
    """Takes in a Series of bag of words, returns the IDF for every term in the bags in a Series."""
    # TODO
    
    IDF_dict = {}
    
    for i in bags:
        for j in i.index:
            counter = 0
            if j not in IDF_dict:
                for k in bags:
                    if j in k.index:
                        counter += 1
                IDF_dict[j] = math.log((len(bags) /counter))
    
    return pd.Series(IDF_dict)

def tf_idf(bag, idf):
    """
    Takes in a bag of words and a Series containing IDF scores, 
    returns tf_idf score for every word in bag as a Series
    """
    # TODO
    tf = term_frequency(bag)
    tf_idf_dict = {}
    
    for i in bag.index:
        tf_idf_dict[i] = idf[i] *tf[i]
    
    return pd.Series(tf_idf_dict)

In [20]:
idf = inverse_document_frequency(business_reviews.bags)
business_reviews['tf_idf'] = business_reviews.bags
business_reviews.tf_idf = business_reviews.tf_idf.map(lambda x: tf_idf(x , idf))
business_reviews.head()

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,business_id,text,categories,text_len,text_tokens,bags,tf_idf
0,JCMnL6-pbWmG7xSVkdBuoQ,"We had the ""hot-n-ready"" supreme and it was a...","[restaurants, pizza]",838,"( , we, had, the, "", hot, -, n, -, ready, "", s...",-PRON- 15 have 1 the ...,-PRON- 0.001484 have 0.000281 th...
1,G3OeHF5jrHvNqi4o3aNezQ,"I love this place! First, it's around the co...","[restaurants, bistros]",14546,"( , i, love, this, place, !, , first, ,, it, ...",i 75 love 1 this 3...,i 0.000403 love 0.000245 th...
2,KSKy6-KujlMkwjxlFvMzJw,Eyes. You need them to see. Shocking statisti...,"[health & medical, optometrists, shopping, eye...",2066,"( , eyes, ., you, need, them, to, see, ., shoc...",eye 3 -PRON- 29 need ...,eye 0.017423 -PRON- 0.001115 ne...
3,DoWLGxBbR24_0r7mpGg__w,First you'll notice there is nobody eating in...,"[restaurants, food, desserts, sandwiches, pizza]",11189,"( , first, you, 'll, notice, there, is, nobody...",first 1 -PRON- 140 notice ...,first 0.000323 -PRON- 0.000952 noti...
4,Xh0GgOn6LjfYJJUZQsjDTg,The hibachi show is the highlight. My family ...,"[japanese, restaurants]",10697,"( , the, hibachi, show, is, the, highlight, .,...",the 132 hibachi 26 show ...,the 0.000573 hibachi 0.069090 sh...


**Word similarities in de text**

We kwam er al snel achter dat een woord met een hoge TF-IDF vaak niet echts iets zeggen over een categorie maar meer over een ervaring die een klant heeft gehad. We zijn daarom alle woorden gaan gebruiken in de text. Als een woord een hoge similarity heeft met een van de categorieen die bestaan, dan word die categorie toegevoegd aan een business.

In [23]:
business_reviews.loc[0, 'tf_idf'].sort_values(ascending=False)

sometimes       0.062583
inconsistent    0.049963
pep             0.041083
doneness        0.041083
somewhat        0.035072
                  ...   
with            0.000509
the             0.000296
have            0.000281
and             0.000236
i               0.000099
Length: 81, dtype: float64

In [26]:
category_list = []

for i in business_reviews.categories:
    for j in i:
        if j not in category_list:
            category_list.append(j)

In [28]:
def put_new_categories(index_id, categories):
    tokens1 = nlp( " ".join(business_reviews.loc[index_id, 'categories']))
    tokens2 = business_reviews.loc[index_id, 'text_tokens']
    old_categories = business_reviews.loc[index_id, 'categories']
#     print(old_categories)
    # print(business_reviews.loc[index_id, 'text'])


    new_categories = []

    for token1 in tokens1:
        for token2 in tokens2:
            if(token1.similarity(token2) >= 0.6):
    #             print(token1.text, token2.text, token1.similarity(token2))
                if(token2.text in category_list and token2.text not in new_categories and token2.text not in old_categories):
                    new_categories.append(token2.text)
                    old_categories.append(token2.text)
#     print(old_categories)

In [33]:
start = time.time()

for i in business_reviews.index:
    put_new_categories(i, category_list)
    
end = time.time()
print(end - start)


295.9202835559845


In [35]:
business_reviews['categories'] = business_reviews['categories'].map(lambda x: '|'.join(x))
business_reviews.head()

,business_id,text,categories,text_len,text_tokens,bags,tf_idf
0,JCMnL6-pbWmG7xSVkdBuoQ,"We had the ""hot-n-ready"" supreme and it was a...",restaurants|pizza,838,"( , we, had, the, "", hot, -, n, -, ready, "", s...",-PRON- 15 have 1 the ...,-PRON- 0.001484 have 0.000281 th...
1,G3OeHF5jrHvNqi4o3aNezQ,"I love this place! First, it's around the co...",restaurants|bistros|food|salad|pizza|seafood|m...,14546,"( , i, love, this, place, !, , first, ,, it, ...",i 75 love 1 this 3...,i 0.000403 love 0.000245 th...
2,KSKy6-KujlMkwjxlFvMzJw,Eyes. You need them to see. Shocking statisti...,health & medical|optometrists|shopping|eyewear...,2066,"( , eyes, ., you, need, them, to, see, ., shoc...",eye 3 -PRON- 29 need ...,eye 0.017423 -PRON- 0.001115 ne...
3,DoWLGxBbR24_0r7mpGg__w,First you'll notice there is nobody eating in...,restaurants|food|desserts|sandwiches|pizza|sal...,11189,"( , first, you, 'll, notice, there, is, nobody...",first 1 -PRON- 140 notice ...,first 0.000323 -PRON- 0.000952 noti...
4,Xh0GgOn6LjfYJJUZQsjDTg,The hibachi show is the highlight. My family ...,japanese|restaurants|food|soup|seafood|salad|s...,10697,"( , the, hibachi, show, is, the, highlight, .,...",the 132 hibachi 26 show ...,the 0.000573 hibachi 0.069090 sh...


In [36]:
def extract_categories(business):
    """Create an unfolded genre dataframe. Unpacks genres seprated by a '|' into seperate rows.

    Arguments:
    movies -- a dataFrame containing at least the columns 'movieId' and 'genres' 
              where genres are seprated by '|'
    """
    genres_m = business.apply(lambda row: pd.Series([row['business_id']] + row['categories'].lower().split("|")), axis=1)
    stack_genres = genres_m.set_index(0).stack()
    df_stack_genres = stack_genres.to_frame()
    df_stack_genres['business_id'] = stack_genres.index.droplevel(1)
    df_stack_genres.columns = ['category', 'business_id']
    return df_stack_genres.reset_index()[['business_id', 'category']]

def pivot_genres(df):
    """Create a one-hot encoded matrix for genres.
    
    Arguments:
    df -- a dataFrame containing at least the columns 'movieId' and 'genre'
    
    Output:
    a matrix containing '0' or '1' in each cell.
    1: the movie has the genre
    0: the movie does not have the genre
    """
    return df.pivot_table(index = 'business_id', columns = 'category', aggfunc = 'size', fill_value=0)

In [37]:
df_categories = extract_categories(business_reviews)
df_categories.head()

,business_id,category
0,JCMnL6-pbWmG7xSVkdBuoQ,restaurants
1,JCMnL6-pbWmG7xSVkdBuoQ,pizza
2,G3OeHF5jrHvNqi4o3aNezQ,restaurants
3,G3OeHF5jrHvNqi4o3aNezQ,bistros
4,G3OeHF5jrHvNqi4o3aNezQ,food


**Utility en similarity matrix**

Aan de hand van de nieuw toegevoegde categorien maken een utility matrix en berekenen we aan de hand van de Jaccard index een similarity matrix

In [38]:
df_utility_categories = pivot_genres(df_categories)
display(df_utility_categories.head())

category,,accessories,active life,acupuncture,american (new),american (traditional),amusement parks,apartments,appliances,arts & crafts,...,water parks,waxing,wheel & rim repair,wigs,windows installation,wine & spirits,wine bars,wineries,women's clothing,yoga
business_id,,,,,,,,,,,,,,,,,,,,,
-2mTrGw66Sg-ipomugRlmw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-9Tfvyk5ip565mhYBs0zMQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
-JXt_PElFkCvgRViW05bew,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-SzaOJH2Zi1soNWfGDBSAw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-atM342fpLyVYlTDArNqaQ,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [39]:
def create_similarity_matrix_jaccard(matrix):
    m11 = matrix @ matrix.T
    m10 = pd.DataFrame(matrix.sum(axis = 1).values + np.zeros(m11.shape), index = m11.index, columns = m11.index)
    m01 = m10.T
    return m11/(m01 + m10 - m11)

In [40]:
df_similarity_categories = create_similarity_matrix_jaccard(df_utility_categories)
df_similarity_categories

business_id,-2mTrGw66Sg-ipomugRlmw,-9Tfvyk5ip565mhYBs0zMQ,-JXt_PElFkCvgRViW05bew,-SzaOJH2Zi1soNWfGDBSAw,-atM342fpLyVYlTDArNqaQ,-dDadUS-WWxOo94KNP73zg,-tCvDzWk1hLk0VqxsWehGA,-tdTHBQDGEVp4VBGVPc6dA,0NDbUCHi9YsRwgG3iZO8Kg,0PZjd1qdID11kyis2l3RkQ,...,yuxbg-3pqexbJuXCvUDfxw,yysIFlakVnCr9JEob_lccw,z2Qsl-X1jjtoaVOb46iHlw,zRGCIIez1z4ve5Hppa7eVQ,zZ4NHMRY9gB5UFapV7OPIQ,zZnGiUCLdz8wuc7Klq4YuQ,zeyjXnME-Op6LnAMkjkrCQ,zoiYC_2OLaHuSuZG5_WPfw,zwya8nELRKnNV3Nc1yLfpA,zysuthY02TaG7xvy-Lceew
business_id,,,,,,,,,,,,,,,,,,,,,
-2mTrGw66Sg-ipomugRlmw,1.000000,0.000000,0.157895,0.166667,0.000000,0.153846,0.266667,0.0,0.263158,0.125000,...,0.176471,0.200000,0.066667,0.312500,0.058824,0.000000,0.000,0.153846,0.047619,0.142857
-9Tfvyk5ip565mhYBs0zMQ,0.000000,1.000000,0.000000,0.083333,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000,0.000000,0.000000,0.000000
-JXt_PElFkCvgRViW05bew,0.157895,0.000000,1.000000,0.055556,0.000000,0.083333,0.133333,0.0,0.222222,0.230769,...,0.285714,0.100000,0.000000,0.117647,0.000000,0.000000,0.000,0.181818,0.000000,0.166667
-SzaOJH2Zi1soNWfGDBSAw,0.166667,0.083333,0.055556,1.000000,0.071429,0.090909,0.142857,0.0,0.105263,0.071429,...,0.062500,0.235294,0.083333,0.125000,0.071429,0.062500,0.000,0.090909,0.055556,0.083333
-atM342fpLyVYlTDArNqaQ,0.000000,0.111111,0.000000,0.071429,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.090909,0.166667,0.125,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zZnGiUCLdz8wuc7Klq4YuQ,0.000000,0.090909,0.000000,0.062500,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.076923,1.000000,0.100,0.000000,0.000000,0.000000
zeyjXnME-Op6LnAMkjkrCQ,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.500000,0.100000,1.000,0.000000,0.000000,0.000000
zoiYC_2OLaHuSuZG5_WPfw,0.153846,0.000000,0.181818,0.090909,0.000000,0.200000,0.250000,0.0,0.153846,0.285714,...,0.222222,0.153846,0.000000,0.200000,0.000000,0.000000,0.000,1.000000,0.000000,0.400000


In [41]:
def pivot_ratings(df):
    """Creates a utility matrix for user ratings for movies
    
    Arguments:
    df -- a dataFrame containing at least the columns 'movieId' and 'genres'
    
    Output:
    a matrix containing a rating in each cell. np.nan means that the user did not rate the movie
    """
    return df.pivot_table(values='stars', columns='user_id', index='business_id')






def predict_ratings(similarity, utility, to_predict):
    """Predicts the predicted rating for the input test data.
    
    Arguments:
    similarity -- a dataFrame that describes the similarity between items
    utility    -- a dataFrame that contains a rating for each user (columns) and each movie (rows). 
                  If a user did not rate an item the value np.nan is assumed. 
    to_predict -- A dataFrame containing at least the columns movieId and userId for which to do the predictions
    """
    # copy input (don't overwrite)
    ratings_test_c = to_predict.copy()
    # apply prediction to each row
    ratings_test_c['predicted rating'] = to_predict.apply(lambda row: predict_ids(similarity, utility, row['user_id'], row['business_id']), axis=1)
    return ratings_test_c

### Helper functions for predict_ratings_item_based ###

def predict_ids(similarity, utility, userId, itemId):
    # select right series from matrices and compute
    if userId in utility.columns and itemId in similarity.index:
        return predict_vectors(utility.loc[:,userId], similarity[itemId])
    return 0

def predict_vectors(user_ratings, similarities):
    # select only movies actually rated by user
    relevant_ratings = user_ratings.dropna()
    
    # select corresponding similairties
    similarities_s = similarities[relevant_ratings.index]
    
    # select neighborhood
    similarities_s = similarities_s[similarities_s > 0.0]
    relevant_ratings = relevant_ratings[similarities_s.index]
    
    # if there's nothing left return a prediction of 0
    norm = similarities_s.sum()
    if(norm == 0):
        return 0
    
    # compute a weighted average (i.e. neighborhood is all) 
    return np.dot(relevant_ratings, similarities_s)/norm

def mse(predicted_ratings):
    """Computes the mean square error between actual ratings and predicted ratings
    
    Arguments:
    predicted_ratings -- a dataFrame containing the columns rating and predicted rating
    """
    diff = predicted_ratings['stars'] - predicted_ratings['predicted rating']
    return (diff**2).mean()

In [42]:
df_utility_ratings = pivot_ratings(reviews)
df_utility_ratings

user_id,-0x2ov-qcCopv32Imm-TYg,-19x1bavUQaF7o_gGu_fVw,-1IGzw3QBZgCo-868DGyqA,-4KfpGWfDOt8JP1u1fmtQA,-4L6vSCT4W2jvMgvYH4woA,-5psN9APmb8RcprBFA6lfw,-9-M4wWNz4DDxEAn7e7QNA,-AKaQlc45sdfUD2J2H6cew,-CjqaUZvigXTKPq0HvIk8w,-Dc_ppG3yHPWrksKyPcFAQ,...,zgTFkLtvDqLkv7Lnepjm2Q,zjmrDsQwwSpdh5Ciozd-0A,zlmGyk3UeyPt0LddUCMWmA,znQ36nMZ3uMnnXyOU0I55Q,zox4l3TDP_xAcAQ_NsklYg,zpMvXReWlU4sKjY8mNW_QQ,zqDAWloU0XpjJoNJzH9ORg,zsNbI-SLWo6t9bSg4d6P1Q,zt1Kcygrq-m6I-ztJ3dtCw,zyg4-MFtfPWmwucVazSjfw
business_id,,,,,,,,,,,,,,,,,,,,,
-2mTrGw66Sg-ipomugRlmw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9Tfvyk5ip565mhYBs0zMQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-JXt_PElFkCvgRViW05bew,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,2.0,NaN,NaN,NaN
-SzaOJH2Zi1soNWfGDBSAw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-atM342fpLyVYlTDArNqaQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zZnGiUCLdz8wuc7Klq4YuQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zeyjXnME-Op6LnAMkjkrCQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zoiYC_2OLaHuSuZG5_WPfw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Predictions**

Met dit algoritme kunnen 635 van de 1269 ratings voorspeld worden met een MSE van 2.142652199471348.
Wij vinden dit erg weinig ratings en zullen daarom de algortime moeten verbeteren om meer voorspellingen te kunnen genereren.

In [49]:
df_sim = predict_ratings(df_similarity_categories, df_utility_ratings, reviews_test[['user_id', 'business_id', 'stars']])
df_sim

,user_id,business_id,stars,predicted rating
1,riBEhEli9jk44uJhQdhLWQ,Vr8IDPmTZ7SbnsKy6pV3PA,5.0,0.000000
3,DkBEcLE7gsGJh6pjyOzKAg,d-u-4Y0BJpbKnEd2NtXAHA,5.0,0.000000
6,fBb4Y610RS903w-nTbatcw,A5AeXTmOjLbaLPHcfRkmGg,5.0,0.000000
14,pyCEuueMTMWS0Qzw-qe63w,vc0v-vOIwWpa8eA9Fcc2zQ,4.0,2.680291
27,y05H_1uQhO4frLTJ5_tGCQ,UuHT4mGH2E0cFrG-RaYfaw,3.0,4.067797
...,...,...,...,...
5008,8Oe30JkZqwuhBULHa96xlg,aZbXAxSSvvlJIF-I9KBhOA,5.0,0.000000
5016,S6gAeiubzQ7FJOdMvipsog,Fxk23pD0vgRa95u1mPQ4tQ,3.0,3.027051
5019,l3niK3vPnjfq-hCfe-ewjg,qGtagj4xXNrF9rIEkiihTw,5.0,5.000000
5022,2AXgvV9-QNcuw58rZn4w9g,oWwa7fZCUdjsyhAuUGXtxQ,4.0,0.000000


In [50]:
mse(df_sim)

8.319609256630656

In [51]:
df_sim = df_sim[df_sim["predicted rating"] > 0]
df_sim

,user_id,business_id,stars,predicted rating
14,pyCEuueMTMWS0Qzw-qe63w,vc0v-vOIwWpa8eA9Fcc2zQ,4.0,2.680291
27,y05H_1uQhO4frLTJ5_tGCQ,UuHT4mGH2E0cFrG-RaYfaw,3.0,4.067797
33,R2U5IYRAbtOJGpshsiypTg,gAOzhWPFHBJi0L_QEkAv8g,3.0,3.958748
43,zUJ3fHdJG2zv0Q3CtSyO1Q,cfduvIw7CGD8KjMa1gqkaw,4.0,4.560976
50,_DiARIy42guR5sle_KUbNg,DoWLGxBbR24_0r7mpGg__w,2.0,3.137056
...,...,...,...,...
5004,GOhjRhAyVj3Ir7LQ38Bj5Q,PO2ztFbgAMsf_F0TxjKwNQ,1.0,1.000000
5007,_41OzxkPyg5NnuwPagYBGg,PYvRVwP-XUY6eQyg9YZ4TA,4.0,4.333333
5016,S6gAeiubzQ7FJOdMvipsog,Fxk23pD0vgRa95u1mPQ4tQ,3.0,3.027051
5019,l3niK3vPnjfq-hCfe-ewjg,qGtagj4xXNrF9rIEkiihTw,5.0,5.000000


In [52]:
mse(df_sim)

2.142652199471348